In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('pracitce-364603-f0aa0101f81d.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#date = 0 #当日は0、前日は1・・・
#
date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 21018019 # 店舗ID
SPREADSHEET_KEY = '1eGaMLKhXpFPulu2MPBI_k_wlkvMLSjcRSUklfNl5brc' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 595 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p40row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
#s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p40table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "40":
                    p40table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "40":
                        p40table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.4RAW'!F" + str(int(p40row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p40table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/19', '025533', '1', '21026.0', '-3309.0', '23', '9']
['08/19', '025409', '2', '25897.0', '7223.0', '92', '8']
['08/19', '025785', '3', '18974.0', '2107.0', '26', '12']
['08/19', '025523', '4', '26410.0', '-1504.0', '27', '11']
['08/19', '025704', '5', '19744.0', '-4513.0', '23', '8']
['08/19', '025471', '6', '19487.0', '1506.0', '21', '7']
['08/19', '025774', '7', '31538.0', '8126.0', '57', '12']
['08/19', '025847', '8', '30256.0', '-21666.0', '20', '9']
['08/19', '025533', '9', '35897.0', '2107.0', '52', '20']
['08/19', '025533', '10', '29744.0', '18658.0', '64', '10']
['08/19', '025712', '11', '14872.0', '-1203.0', '28', '10']
['08/19', '025698', '12', '16667.0', '1205.0', '26', '9']
['08/19', '025409', '13', '22051.0', '-11133.0', '17', '5']
['08/19', '025605', '14', '25897.0', '1205.0', '29', '15']
['08/19', '025601', '15', '21282.0', '-5416.0', '17', '6']
['08/19', '025722', '16', '25128.0', '7524.0', '42', '8']
['08/19', '025892', '17', '25385.0', '1205.0', '31', '11']
['08/

['08/19', '025803', '140', '24103.0', '5719.0', '17', '4']
['08/19', '025737', '141', '38205.0', '-20763.0', '11', '1']
['08/19', '025483', '142', '1795.0', '-1504.0', '0', '0']
['08/19', '025856', '143', '50513.0', '-16249.0', '18', '5']
['08/19', '025497', '144', '0', '0', '0', '0']
['08/19', '025750', '145', '19744.0', '-9930.0', '24', '4']
['08/19', '025798', '146', '10000.0', '3010.0', '10', '3']
['08/19', '025856', '147', '35385.0', '-11434.0', '12', '3']
['08/19', '025791', '148', '13333.0', '26482.0', '24', '2']
['08/19', '025531', '149', '4872.0', '3311.0', '7', '1']
['08/19', '025737', '150', '20000.0', '603.0', '32', '4']
['08/19', '025825', '151', '22821.0', '43033.0', '40', '5']
['08/19', '025779', '152', '16154.0', '1506.0', '17', '5']
['08/19', '025824', '153', '26667.0', '-7823.0', '18', '3']
['08/19', '025795', '154', '14872.0', '-4513.0', '12', '5']
['08/19', '025864', '155', '8462.0', '21066.0', '35', '2']
['08/19', '025483', '156', '23846.0', '15649.0', '28', '5']
[

['08/19', '025585', '280', '769.0', '-1203.0', '0', '0']
['08/19', '025789', '281', '3077.0', '-2406.0', '1', '1']
['08/19', '025497', '282', '3590.0', '-1504.0', '1', '1']
['08/19', '025649', '283', '13333.0', '2107.0', '9', '1']
['08/19', '025852', '284', '0', '0', '0', '0']
['08/19', '025796', '285', '10256.0', '-7221.0', '1', '1']
['08/19', '025801', '286', '3333.0', '-2707.0', '0', '0']
['08/19', '025829', '287', '5897.0', '-1504.0', '2', '1']
['08/19', '025807', '288', '8974.0', '-3911.0', '5', '2']
['08/19', '025792', '289', '8718.0', '-4814.0', '2', '2']
['08/19', '025792', '290', '9231.0', '-6920.0', '1', '1']
['08/19', '025792', '291', '5385.0', '-5115.0', '0', '0']
['08/19', '025680', '292', '1282.0', '-1203.0', '0', '0']
['08/19', '025680', '293', '7436.0', '17756.0', '16', '3']
['08/19', '025680', '294', '0', '0', '0', '0']
['08/19', '025750', '295', '19231.0', '-16851.0', '0', '0']
['08/19', '025750', '296', '4872.0', '-1203.0', '9', '1']
['08/19', '025750', '297', '29231

['08/19', '025838', '421', '13590.0', '2107.0', '11', '3']
['08/19', '025838', '422', '14359.0', '2408.0', '11', '4']
['08/19', '025838', '423', '13077.0', '12339.0', '20', '2']
['08/19', '025687', '424', '4103.0', '-2406.0', '1', '1']
['08/19', '025687', '425', '18718.0', '24978.0', '31', '31']
['08/19', '025687', '426', '4872.0', '-3008.0', '1', '1']
['08/19', '025687', '427', '9744.0', '-7823.0', '0', '0']
['08/19', '025687', '428', '13590.0', '-5416.0', '4', '4']
['08/19', '025687', '429', '19744.0', '-11133.0', '4', '4']
['08/19', '025687', '430', '12308.0', '-10531.0', '0', '0']
['08/19', '025687', '431', '24872.0', '-15647.0', '5', '5']
['08/19', '025687', '432', '25128.0', '-6920.0', '13', '13']
['08/19', '025670', '433', '13077.0', '-1504.0', '26', '2']
['08/19', '025670', '434', '14872.0', '-10231.0', '2', '2']
['08/19', '025670', '435', '17692.0', '-9027.0', '6', '3']
['08/19', '025670', '436', '4615.0', '-2707.0', '1', '1']
['08/19', '025670', '437', '16667.0', '-6920.0', '

['08/19', '025856', '561', '20256.0', '5117.0', '15', '3']
['08/19', '025856', '562', '23077.0', '-16851.0', '2', '1']
['08/19', '025856', '563', '13077.0', '8427.0', '13', '2']
['08/19', '025856', '564', '29231.0', '-6920.0', '12', '3']
['08/19', '025856', '565', '12051.0', '-1203.0', '6', '3']
['08/19', '025856', '566', '18974.0', '8728.0', '16', '2']
['08/19', '025856', '567', '24359.0', '1806.0', '15', '4']
['08/19', '025856', '568', '18974.0', '2107.0', '12', '5']
['08/19', '025856', '569', '28718.0', '-7522.0', '11', '4']
['08/19', '025856', '570', '32821.0', '-3309.0', '16', '5']
['08/19', '025864', '571', '0', '0', '0', '0']
['08/19', '025864', '572', '1795.0', '-1504.0', '0', '0']
['08/19', '025864', '573', '0', '0', '0', '0']
['08/19', '025864', '574', '0', '0', '0', '0']
['08/19', '025864', '575', '4615.0', '15950.0', '22', '1']
['08/19', '025864', '576', '8462.0', '4515.0', '16', '1']
['08/19', '025665', '577', '41282.0', '-24374.0', '7', '4']
['08/19', '025665', '578', '47

['08/19', '120145', '698', '2753', '-926.0', '0', '5', '21']
['08/19', '120145', '699', '6173', '-972.0', '0', '13', '56']
['08/19', '120145', '700', '8997', '139.0', '0', '15', '88']
['08/19', '120145', '701', '5062', '-1944.0', '0', '9', '36']
['08/19', '120145', '702', '6796', '-1944.0', '0', '14', '52']
['08/19', '120103', '703', '1985', '4769.0', '45', '3', '6']
['08/19', '120103', '704', '2233', '-2083.0', '3', '1', '6']
['08/19', '120103', '705', '3328', '-1389.0', '16', '5', '11']
['08/19', '120103', '706', '3386', '6898.0', '67', '3', '10']
['08/19', '120103', '707', '2386', '278.0', '27', '1', '8']
['08/19', '120103', '708', '3630', '-1944.0', '14', '6', '11']
['08/19', '120103', '709', '1517', '-880.0', '8', '2', '6']
['08/19', '120103', '710', '1921', '417.0', '21', '2', '6']
['08/19', '120103', '711', '2465', '-2037.0', '12', '2', '10']
['08/19', '120140', '712', '4890', '-3426.0', '0', '1', '34']
['08/19', '120140', '713', '6170', '-2315.0', '0', '1', '61']
['08/19', '120

['08/19', '119916', '830', '4756', '-1019.0', '16', '9', '--']
['08/19', '119916', '831', '2254', '-231.0', '6', '7', '--']
['08/19', '119916', '832', '6146', '2130.0', '31', '17', '--']
['08/19', '119916', '833', '2788', '-972.0', '6', '10', '--']
['08/19', '119916', '834', '6223', '556.0', '25', '16', '--']
['08/19', '119916', '835', '2593', '-1296.0', '7', '3', '--']
['08/19', '119916', '836', '5555', '-139.0', '22', '14', '--']
['08/19', '119916', '837', '5443', '-1991.0', '16', '10', '--']
['08/19', '119916', '838', '2265', '-1111.0', '4', '7', '--']
['08/19', '119916', '839', '5641', '-1204.0', '17', '14', '--']
['08/19', '119916', '840', '8978', '2593.0', '40', '31', '--']
['08/19', '119916', '841', '2691', '-694.0', '7', '5', '--']
['08/19', '119916', '842', '3979', '417.0', '17', '9', '--']
['08/19', '119916', '843', '8294', '-231.0', '33', '14', '--']
['08/19', '119916', '844', '2486', '-278.0', '9', '5', '--']
['08/19', '119916', '845', '2959', '-833.0', '8', '5', '--']
['08

['08/19', '119913', '963', '1233', '-463.0', '3', '2', '--']
['08/19', '119913', '964', '2986', '-278.0', '9', '9', '--']
['08/19', '119913', '965', '3470', '2037.0', '22', '11', '--']
['08/19', '119913', '966', '4281', '139.0', '18', '5', '--']
['08/19', '119913', '967', '3410', '-231.0', '13', '8', '--']
['08/19', '119913', '968', '3879', '1204.0', '20', '10', '--']
['08/19', '119913', '969', '0', '0', '0', '0', '--']
['08/19', '119913', '970', '3395', '-1157.0', '8', '4', '--']
['08/19', '119913', '971', '4222', '-370.0', '15', '13', '--']
['08/19', '119913', '972', '1565', '-972.0', '2', '3', '--']
['08/19', '119913', '973', '799', '1111.0', '7', '6', '--']
['08/19', '119913', '974', '3513', '5324.0', '34', '21', '--']
['08/19', '119913', '975', '915', '-93.0', '4', '2', '--']
['08/19', '119913', '976', '3266', '-370.0', '11', '7', '--']
['08/19', '119913', '977', '1727', '139.0', '8', '1', '--']
['08/19', '119913', '978', '1030', '-417.0', '2', '0', '--']
['08/19', '120110', '979'

['08/19', '120073', '1095', '8563', '3704.0', '0', '23', '70']
['08/19', '120073', '1096', '8194', '2685.0', '0', '35', '66']
['08/19', '119913', '1097', '2479', '-972.0', '7', '1', '--']
['08/19', '119913', '1098', '5070', '-1250.0', '15', '11', '--']
['08/19', '119913', '1099', '2545', '-231.0', '10', '2', '--']
['08/19', '119913', '1100', '3871', '-3009.0', '3', '3', '--']
['08/19', '119913', '1101', '3309', '-741.0', '9', '9', '--']
['08/19', '119913', '1102', '3800', '3565.0', '31', '8', '--']
['08/19', '119913', '1103', '3450', '-1065.0', '9', '6', '--']
['08/19', '119913', '1104', '4185', '4259.0', '35', '7', '--']
['08/19', '119913', '1105', '2859', '-370.0', '12', '1', '--']
['08/19', '119913', '1106', '2208', '-509.0', '6', '6', '--']
['08/19', '119913', '1107', '5823', '-139.0', '21', '17', '--']
['08/19', '119913', '1108', '4232', '-231.0', '15', '9', '--']
['08/19', '119913', '1109', '0', '0', '0', '0', '--']
['08/19', '119913', '1110', '3045', '-2037.0', '3', '6', '--']
[

{'spreadsheetId': '1eGaMLKhXpFPulu2MPBI_k_wlkvMLSjcRSUklfNl5brc',
 'updates': {'spreadsheetId': '1eGaMLKhXpFPulu2MPBI_k_wlkvMLSjcRSUklfNl5brc',
  'updatedRange': 'S20RAW!F24638:M25136',
  'updatedRows': 499,
  'updatedColumns': 8,
  'updatedCells': 3992}}

In [3]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p40table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "40":
                    p40table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "40":
                        p40table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

#wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
#wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
#wb.values_append("'P0.4RAW'!F" + str(int(p40row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p40table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['07/29', '025533', '1', '39231.0', '-9930.0', '35', '17']
['07/29', '025409', '2', '37179.0', '3913.0', '99', '15']
['07/29', '025715', '3', '22051.0', '-10231.0', '24', '9']
['07/29', '025523', '4', '32821.0', '-16550.0', '15', '13']
['07/29', '025704', '5', '16410.0', '-5115.0', '15', '9']
['07/29', '025471', '6', '23077.0', '-3008.0', '18', '9']
['07/29', '025544', '7', '15385.0', '7223.0', '67', '7']
['07/29', '025847', '8', '34872.0', '-19258.0', '52', '15']
['07/29', '025533', '9', '36154.0', '4214.0', '57', '11']
['07/29', '025533', '10', '38718.0', '-902.0', '52', '16']
['07/29', '025712', '11', '15897.0', '-2406.0', '31', '10']
['07/29', '025698', '12', '18718.0', '-6018.0', '11', '6']
['07/29', '025409', '13', '27949.0', '12339.0', '99', '16']
['07/29', '025605', '14', '6410.0', '-1504.0', '5', '4']
['07/29', '025601', '15', '30000.0', '5117.0', '48', '12']
['07/29', '025572', '16', '26154.0', '-3008.0', '24', '11']
['07/29', '025759', '17', '16923.0', '2709.0', '22', '6']
[

['07/29', '025803', '140', '24103.0', '-11133.0', '6', '3']
['07/29', '025737', '141', '31282.0', '-1805.0', '41', '5']
['07/29', '025483', '142', '27179.0', '-10231.0', '9', '4']
['07/29', '025638', '143', '26410.0', '-1203.0', '18', '6']
['07/29', '025497', '144', '8205.0', '9931.0', '11', '2']
['07/29', '025750', '145', '28974.0', '6320.0', '61', '3']
['07/29', '025798', '146', '33077.0', '-2106.0', '21', '12']
['07/29', '025800', '147', '16410.0', '2107.0', '13', '4']
['07/29', '025791', '148', '9744.0', '-7221.0', '1', '1']
['07/29', '025531', '149', '16667.0', '-5717.0', '7', '3']
['07/29', '025737', '150', '51026.0', '-2406.0', '70', '7']
['07/29', '025825', '151', '28462.0', '-9930.0', '10', '6']
['07/29', '025779', '152', '23333.0', '10232.0', '28', '5']
['07/29', '025803', '153', '20000.0', '-14744.0', '1', '1']
['07/29', '025795', '154', '13590.0', '13844.0', '23', '3']
['07/29', '025864', '155', '29231.0', '-20763.0', '1', '1']
['07/29', '025483', '156', '29744.0', '7524.0'

['07/29', '025313', '279', '24615.0', '-11434.0', '10', '5']
['07/29', '025585', '280', '1538.0', '-1203.0', '0', '0']
['07/29', '025789', '281', '6410.0', '-1203.0', '4', '2']
['07/29', '025497', '282', '17692.0', '-5416.0', '5', '5']
['07/29', '025649', '283', '513.0', '-902.0', '0', '0']
['07/29', '025852', '284', '8462.0', '12941.0', '35', '3']
['07/29', '025796', '285', '12564.0', '2709.0', '10', '3']
['07/29', '025801', '286', '8205.0', '-6319.0', '1', '1']
['07/29', '025829', '287', '27949.0', '-902.0', '14', '6']
['07/29', '025807', '288', '13333.0', '-2106.0', '11', '4']
['07/29', '025792', '289', '8462.0', '3010.0', '7', '1']
['07/29', '025792', '290', '13333.0', '-1203.0', '7', '3']
['07/29', '025792', '291', '6154.0', '-1203.0', '3', '1']
['07/29', '025680', '292', '18718.0', '-6319.0', '7', '2']
['07/29', '025680', '293', '769.0', '-1203.0', '0', '0']
['07/29', '025680', '294', '5897.0', '1205.0', '4', '2']
['07/29', '025750', '295', '16154.0', '-9328.0', '18', '2']
['07/2

['07/29', '025838', '418', '3846.0', '-2406.0', '1', '1']
['07/29', '025838', '419', '22051.0', '-1203.0', '16', '5']
['07/29', '025838', '420', '9231.0', '-5115.0', '4', '3']
['07/29', '025838', '421', '18462.0', '6621.0', '17', '2']
['07/29', '025838', '422', '11282.0', '-3309.0', '5', '3']
['07/29', '025838', '423', '25897.0', '19260.0', '35', '6']
['07/29', '025687', '424', '9231.0', '2709.0', '7', '7']
['07/29', '025687', '425', '3846.0', '3311.0', '4', '4']
['07/29', '025687', '426', '18462.0', '16251.0', '27', '27']
['07/29', '025687', '427', '2821.0', '-2406.0', '0', '0']
['07/29', '025687', '428', '5897.0', '3311.0', '7', '7']
['07/29', '025687', '429', '2308.0', '-1805.0', '0', '0']
['07/29', '025687', '430', '0', '0', '0', '0']
['07/29', '025687', '431', '30513.0', '12038.0', '28', '28']
['07/29', '025687', '432', '2308.0', '-1805.0', '0', '0']
['07/29', '025670', '433', '37949.0', '3311.0', '50', '11']
['07/29', '025670', '434', '24872.0', '-16249.0', '6', '6']
['07/29', '0

['07/29', '025869', '557', '18462.0', '3311.0', '19', '6']
['07/29', '025869', '558', '32308.0', '23172.0', '50', '7']
['07/29', '025856', '559', '42564.0', '15950.0', '34', '9']
['07/29', '025856', '560', '35641.0', '-10832.0', '13', '7']
['07/29', '025856', '561', '32051.0', '-18055.0', '6', '4']
['07/29', '025856', '562', '46154.0', '-12036.0', '18', '11']
['07/29', '025856', '563', '44103.0', '6621.0', '29', '9']
['07/29', '025856', '564', '35385.0', '-20462.0', '5', '3']
['07/29', '025856', '565', '33590.0', '-14744.0', '9', '5']
['07/29', '025856', '566', '37692.0', '33103.0', '43', '5']
['07/29', '025856', '567', '43846.0', '-11434.0', '17', '6']
['07/29', '025856', '568', '37692.0', '16853.0', '32', '13']
['07/29', '025856', '569', '40513.0', '3612.0', '25', '6']
['07/29', '025856', '570', '38718.0', '-6619.0', '17', '5']
['07/29', '025856', '571', '26410.0', '14144.0', '24', '7']
['07/29', '025856', '572', '40513.0', '7524.0', '28', '6']
['07/29', '025856', '573', '30769.0', '

['07/29', '120122', '727', '6582', '417.0', '70', '19', '--']
['07/29', '120122', '728', '5487', '509.0', '67', '12', '--']
['07/29', '120122', '729', '5292', '-648.0', '54', '12', '--']
['07/29', '120122', '730', '7107', '1296.0', '88', '19', '--']
['07/29', '120122', '731', '6479', '-3056.0', '49', '13', '--']
['07/29', '120122', '732', '5732', '5046.0', '99', '14', '--']
['07/29', '120122', '733', '6573', '-3009.0', '46', '20', '--']
['07/29', '120122', '734', '3965', '7593.0', '93', '8', '--']
['07/29', '120122', '735', '5201', '-2870.0', '35', '8', '--']
['07/29', '120122', '736', '6170', '-2222.0', '51', '14', '--']
['07/29', '120122', '737', '6822', '-3241.0', '48', '21', '--']
['07/29', '120125', '738', '5314', '-602.0', '18', '8', '--']
['07/29', '120125', '739', '8515', '1204.0', '35', '25', '--']
['07/29', '120125', '740', '7975', '1296.0', '32', '23', '--']
['07/29', '120125', '741', '7799', '741.0', '29', '25', '--']
['07/29', '120125', '742', '7641', '2037.0', '38', '15',

['07/29', '120122', '858', '6839', '556.0', '83', '21', '--']
['07/29', '119897', '859', '6611', '-93.0', '50', '0', '28']
['07/29', '119897', '860', '4657', '-2083.0', '33', '0', '12']
['07/29', '119897', '861', '7322', '3148.0', '63', '0', '37']
['07/29', '119897', '862', '5799', '-741.0', '46', '0', '27']
['07/29', '119897', '863', '6175', '463.0', '48', '0', '33']
['07/29', '120034', '864', '5829', '741.0', '0', '9', '49']
['07/29', '120034', '865', '9811', '4352.0', '0', '17', '103']
['07/29', '120034', '866', '7947', '1574.0', '0', '8', '77']
['07/29', '120034', '867', '4650', '-2037.0', '0', '5', '28']
['07/29', '120034', '868', '5534', '-3704.0', '0', '5', '27']
['07/29', '120034', '869', '8935', '370.0', '0', '15', '73']
['07/29', '120034', '870', '4703', '-2685.0', '0', '6', '24']
['07/29', '120034', '871', '6969', '2593.0', '0', '15', '70']
['07/29', '120034', '872', '6560', '-417.0', '0', '8', '48']
['07/29', '120034', '873', '8432', '1204.0', '0', '15', '70']
['07/29', '12

['07/29', '120036', '990', '3899', '-139.0', '13', '10', '--']
['07/29', '120043', '991', '7909', '-1435.0', '23', '24', '--']
['07/29', '120099', '992', '5924', '3519.0', '19', '10', '--']
['07/29', '120036', '993', '2950', '-93.0', '10', '4', '--']
['07/29', '120137', '994', '8179', '-185.0', '23', '21', '44']
['07/29', '120099', '995', '4134', '-787.0', '8', '6', '--']
['07/29', '120080', '996', '7520', '2037.0', '60', '0', '91']
['07/29', '120093', '997', '8137', '-556.0', '57', '0', '57']
['07/29', '120053', '998', '4540', '-2037.0', '0', '11', '44']
['07/29', '120070', '999', '3315', '-2037.0', '0', '10', '17']
['07/29', '120077', '1000', '7670', '2685.0', '0', '29', '69']
['07/29', '120149', '1001', '7103', '-3519.0', '0', '28', '68']
['07/29', '120120', '1002', '8189', '833.0', '0', '0', '69']
['07/29', '120107', '1003', '4561', '1111.0', '17', '0', '36']
['07/29', '120077', '1004', '7366', '-1574.0', '0', '24', '50']
['07/29', '120021', '1005', '4354', '-2593.0', '18', '10', '

{'spreadsheetId': '1eGaMLKhXpFPulu2MPBI_k_wlkvMLSjcRSUklfNl5brc',
 'updates': {'spreadsheetId': '1eGaMLKhXpFPulu2MPBI_k_wlkvMLSjcRSUklfNl5brc',
  'updatedRange': 'S20RAW!F14478:M14941',
  'updatedRows': 464,
  'updatedColumns': 8,
  'updatedCells': 3712}}